# This file is based on the data of hw3
## We mainly do two things:
-  Classify data
- Train the regression model



In [1]:
# check python version
# must be 3.8 to 3.10
from platform import python_version

print(python_version())

3.8.2


In [2]:
# install packages
!pip3 install polygon-api-client --user
!pip3 install sqlalchemy --user
!pip3 install pandas --user
!pip3 install numpy==1.22.3 --user
!pip3 install scikit-learn==0.23.2 --user
!pip3 install pycaret --user

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Read Data Logic
- Read data from csv files in day1 to dataframe
- Sort df by vol and fd
- Label the vol and fd columns by 1, 2, 3

In [3]:
import pandas as pd

In [4]:
# read data from csv
columns = ["id","Mean", "Max", "Min", "VOL", "FD", "r"]
AUDUSD_df = pd.read_csv('./data/AUDUSD_data_points.csv', names=columns, na_values='?', header=None,dtype = 'float64')
EURUSD_df = pd.read_csv('./data/EURUSD_data_points.csv', names=columns, na_values='?', header=None,dtype = 'float64')
GBPEUR_df = pd.read_csv('./data/GBPEUR_data_points.csv', names=columns, na_values='?', header=None,dtype = 'float64')
USDCAD_df = pd.read_csv('./data/USDCAD_data_points.csv', names=columns, na_values='?', header=None,dtype = 'float64')
USDCNY_df = pd.read_csv('./data/USDCNY_data_points.csv', names=columns, na_values='?', header=None,dtype = 'float64')
USDCZK_df = pd.read_csv('./data/USDCZK_data_points.csv', names=columns, na_values='?', header=None,dtype = 'float64')
USDINR_df = pd.read_csv('./data/USDINR_data_points.csv', names=columns, na_values='?', header=None,dtype = 'float64')
USDJPY_df = pd.read_csv('./data/USDJPY_data_points.csv', names=columns, na_values='?', header=None,dtype = 'float64')
USDMXN_df = pd.read_csv('./data/USDMXN_data_points.csv', names=columns, na_values='?', header=None,dtype = 'float64')
USDPLN_df = pd.read_csv('./data/USDPLN_data_points.csv', names=columns, na_values='?', header=None,dtype = 'float64')

In [5]:
# match the currency pairs and data frame
currency_pairs_dict = {"AUDUSD": AUDUSD_df, "EURUSD": EURUSD_df, "GBPEUR": GBPEUR_df, "USDCAD": USDCAD_df,
                       "USDCNY": USDCNY_df, "USDCZK": USDCZK_df, "USDINR": USDINR_df, "USDJPY": USDJPY_df,
                       "USDMXN": USDMXN_df, "USDPLN": USDPLN_df}

In [6]:
# classify the data by vol/FD
# set them as 1=low, 2=medium, 3=high
# first bar=top 33
# second bar=top=67
def classify(df, column_name,first_bar=33, second_bar=67):
    # sort by vol
    df.sort_values(by=[column_name])
    for i, row in df.iterrows():
        if i < first_bar:
            row[column_name] = 1
        elif i < second_bar:
            row[column_name] = 2
        else:
            row[column_name] = 3

In [7]:
# classify all the fds
for df in currency_pairs_dict.values():
    # classify by fd
    classify(df,"FD")
    # classify by vol
    classify(df,"VOL")
    # show the results to check
    print(df)

      id      Mean      Max      Min  VOL   FD         r
0    0.0  0.668007  0.66840  0.66760  1.0  1.0  0.000000
1    1.0  0.667836  0.66830  0.66740  1.0  1.0 -0.000257
2    2.0  0.667750  0.66810  0.66740  1.0  1.0 -0.000128
3    3.0  0.667585  0.66800  0.66720  1.0  1.0 -0.000247
4    4.0  0.667463  0.66800  0.66700  1.0  1.0 -0.000183
5    5.0  0.667559  0.66800  0.66710  1.0  1.0  0.000143
6    6.0  0.667853  0.66830  0.66740  1.0  1.0  0.000441
7    7.0  0.667891  0.66830  0.66750  1.0  1.0  0.000057
8    8.0  0.668069  0.66850  0.66770  1.0  1.0  0.000266
9    9.0  0.667887  0.66840  0.66760  1.0  1.0 -0.000272
10  10.0  0.668272  0.66880  0.66780  1.0  1.0  0.000577
11  11.0  0.668630  0.66900  0.66830  1.0  1.0  0.000537
12  12.0  0.668631  0.66910  0.66820  1.0  1.0  0.000001
13  13.0  0.668585  0.66900  0.66820  1.0  1.0 -0.000069
14  14.0  0.668473  0.66880  0.66810  1.0  1.0 -0.000168
15  15.0  0.668086  0.66880  0.66770  1.0  1.0 -0.000579
16  16.0  0.668514  0.66900  0.

# Train Models

In [9]:
from pycaret import regression

In [17]:
# extract features we want to use to train the model
def extract_features(df):
    df=df[["Mean","FD","VOL","r"]]
    return df

In [12]:
# train model for all currency df
def train_regression_model(df, target):
    regression.setup(data=df,target=target,fold_shuffle=True)
    best=regression.compare_models()
    regression.evaluate_model(best)
    return best

In [20]:
currency_model_dict={}
for curreny, df in currency_pairs_dict.items():
    # extract feature
    df=extract_features(df)
    currency_model_dict[curreny]=train_regression_model(df,'r')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
ridge,Ridge Regression,0.0004,0.0000,0.0005,-0.1224,0.0004,1.3091,0.0050
br,Bayesian Ridge,0.0004,0.0000,0.0005,-0.1230,0.0004,1.3198,0.0050
omp,Orthogonal Matching Pursuit,0.0004,0.0000,0.0005,-0.1233,0.0004,1.3120,0.0050
llar,Lasso Least Angle Regression,0.0004,0.0000,0.0005,-0.1468,0.0004,1.3299,0.0050
dummy,Dummy Regressor,0.0004,0.0000,0.0005,-0.1468,0.0004,1.3299,0.0050
en,Elastic Net,0.0004,0.0000,0.0005,-0.1468,0.0004,1.3299,0.0050
lasso,Lasso Regression,0.0004,0.0000,0.0005,-0.1468,0.0004,1.3299,0.0050
huber,Huber Regressor,0.0004,0.0000,0.0005,-0.1481,0.0004,1.2882,0.0070
lar,Least Angle Regression,0.0004,0.0000,0.0005,-0.1766,0.0004,1.5607,0.0050
lr,Linear Regression,0.0004,0.0000,0.0005,-0.1766,0.0004,1.5607,0.0050


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…